# 离子阱广义 Mølmer-Sørensen 门

*版权所有 (c) 2021 百度量子计算研究所，保留所有权利。*

## 内容概要
本教程中，我们将介绍如何使用量脉在离子阱量子计算中进行广义 Mølmer-Sørensen 门（MS 门）脉冲的生成，并演示如何用广义 Mølmer-Sørensen 门在离子阱量子计算平台中，一步实现多比特的Greenberger–Horne–Zeilinger (GHZ)纠缠态。本教程的大纲如下：


- 离子阱广义 Mølmer-Sørensen 门背景介绍
- 准备工作
- 离子阱广义 Mølmer-Sørensen 门脉冲生成
- 离子阱广义 Mølmer-Sørensen 门生成 GHZ 态
- 总结
- 参考文献

## 离子阱广义 Mølmer-Sørensen 门背景介绍

离子阱量子计算平台和其他平台最大不同之处在于离子之间传递信息的媒介是集体的声子振动模，从而在离子阱中每个量子比特之间可以很容易地通过一些激光脉冲的作用方式联系起来，而无需过多考虑离子之间的距离。这种全连接的特性也给离子阱量子计算带来了独特的好处，我们可以方便地使用激光脉冲生成广义 MS 门，并在此基础上方便地实现多粒子比特的纠缠态——GHZ 态。我们在 [离子阱单双量子比特门](https://quanlse.baidu.com/#/doc/tutorial-ion-trap-single-and-two-qubit-gate) 中介绍了一般的两比特 MS 门。而对于广义的 MS 门，原理较为相近。在多束光速的作用下，M 个被激光作用的离子链总的哈密顿量可以写成：

$$
\hat{H}_{\rm M S}= \hbar \sum_{j}^M \Omega_{j}(t) \hat{{\sigma}}_{x}^{j}\sum_{k=1}^{N}\eta_{j,k}e^{i\mu t}\left(\hat{a}_k e^{-i \omega_{k} t}+\hat{a}_k^{\dagger} e^{i \omega_{k} t}\right)+{\rm H.c.}, 
$$
其中 $j,k$ 分别代表离子和声子的指标，$N$ 是阱中离子总数，也是参与传递信息的声子模个数。$\mu$ 是激光的矢谐，失谐选取和门时间 $t_g$ 一般满足关系 $\mu=2\pi/t_g$。$\hat{\sigma}_x^j$ 是第 $j$ 个离子的 Pauli 矩阵，$\hat a_k^\dagger,\hat a_k$ 分别是第 $k$ 个声子的产生湮灭算符。$\Omega_j(t)$ 是我们需要优化的脉冲。$\eta_{j,k}$ 是振动声子模的 Lamb-Dicke 系数，$\omega_k$ 代表声子的振动频率。为了后续方便，我们设 $\hbar=1$。

在 Lamb-Dicke 参数远远小于1时，上述哈密顿量的 Magnus 二阶展开得到系统的演化算符为：

$$
U(t_g)=\exp \left[-i\sum_{j}^M \hat{\sigma}_{x}^{j}\sum_{k=1}^{N}\left(\alpha_{j, k}(t_g) \hat{a}_{k}^{\dagger}-\alpha_{j, k}^{*}(t_g) \hat{a}_{k}\right) +i\sum_{m<n}^M\chi_{m,n}(t_g)\hat{\sigma}_{x}^{m}\hat{\sigma}_{x}^{n}\right], 
$$
其中 $\alpha_{j, k}(t_g)=\int_0^{t_g} dt f(\eta_{k,j},\Omega_j(t),\omega_k)$，$\chi_{m,n}(t_g)=\sum_k\int_0^{t_g} dt\int_0^{t} dt_1 g(\eta_{k,j},\Omega_m(t), \Omega_n(t_1), \omega_k)$。

第一项代表了离子和声子的耦合，第二项代表了离子与离子之间的耦合。为了获取所有离子之间的最大纠缠，我们需要保证 $\alpha_{j, k}(t_g)\to 0, \chi_{m,n}(t_g)\to \pi/4$，当酉算符公式中 $\{m, n\}$ 大于 1 对时，我们将所形成的门称为广义的 MS 门。

## 准备工作

首先，在成功安装量脉后，用户需要从量脉（Quanlse）和其它常用的 Python 库导入以下模块：

In [ ]:
# Import the necessary packages
from Quanlse.remoteOptimizer import remoteIonGeneralMS as pulseGeneralMS
from Quanlse.Utils import Plot

import sys
import numpy as np
from math import pi

在使用量脉云服务之前，用户需要从 http://quantum-hub.baidu.com 获取一个 token，并使用以下命令将任务提交到量脉的服务器上：

In [ ]:
from Quanlse import Define

Define.hubToken = ""

## 离子阱广义 Mølmer-Sørensen 门脉冲生成


在导入相关包以及云端服务后，我们介绍如何使用量脉离子阱生成广义 MS 门所需的激光脉冲，并演示四比特 GHZ 态的脉冲波形。我们需要设置线性离子阱量子比特的系统参数，包括离子的种类（质量）`mass`、一维阱中量子比特的数目 `ionNumber`、束缚阱的参数 `omega` 以及用于量子比特之间通信的声子模式 `phononMode`，共同组成参数 `args1`：

In [ ]:
# Define the system information
ionNumber = 7  # System qubit number
mass = 171  # System ion mass
omegaXY = 2 * pi * 2e6  # XY direction trap potential frequency
omegaZ = 2 * pi * 0.2e6  # Z direction trap potential frequency, usually omegaZ << omegaXY
phononMode = "transverse"  # Phonon mode which users can choose "transverse" or "axial" 

# Combine the parameter in Tuple
args1 = (ionNumber, mass, omegaXY, omegaZ, phononMode)

同时，我们也要设定广义 MS 门中激光脉冲相关的参数，包括门时间 `tgate`、激光失谐量 `mu`、脉冲切片的数目 `N`，共同组成 `args2`：

In [ ]:
# Define the gate information
tgate = 200  # Total time of quantum gate
mu = 2 * pi / tgate  # The laser detuning, usually related with gate time. but can tuning around 2 * pi / tgate
N = 35  # The pulse sequence slice number, usually N > 3 * ionNumber
dt = tgate / N  # Sample time

# Combine the parameter in Tuple
args2 = (N, dt, mu)

因为离子阱中比特之间全连通，广义 MS 门上需要用户指定激光脉冲作用的量子比特对，即用户需要设置上述酉算符中 $\chi_{m,n}(t_g)$ 不为 0 的指标 $\{[m,n]\}$ 形成的序列 `gatePair`。对于并行的 MS 门，我们可以用如下方式定义成对的参数：

In [ ]:
# Define the gate pair of general Molmer-Sorensen gate
gatePair = ([0, 1], [2, 3], [4, 5])

通过调用 `pulseGeneralMS()` 函数，我们可以生成上述定义的离子对形成的广义 MS 门：


In [ ]:
res, ureal = pulseGeneralMS(gatePair, args1=args1, args2=args2)
print("The generated unitary gate is:\n", np.round(ureal, 2))

通过量脉离子阱，用户可以计算激光脉冲形成广义 MS 门时的门失真度：

In [ ]:
# Print the infidelity of general MS gate
print(f"The parallel Mølmer-Sørensen gate infidelity:\n {res['infidelity']}")

同时用户也可以方便地可视化每个离子对上的脉冲序列：

In [ ]:
pulse = res['pulse_list']
ionpair = gatePair.index([4, 5])  # Choose the pulse sequence of ionpair
Plot.plotPulse([np.arange(N) * dt * (N+1) / N], [pulse[ionpair]],
               title=[f'Pulse for ionpair={gatePair[ionpair]} '],
               xLabel=r'Time ($\mu$s)', yLabel=['Rabi frequency (a.u)'], color=['blue'])


## 离子阱广义 Mølmer-Sørensen 门生成 GHZ 态

广义 MS 门的好处是可以非常方便地生成多比特的 GHZ 态。我们以四比特 GHZ 态为例，说明如何用量脉离子阱生成四比特的 GHZ 态。在离子阱的酉算符 $U(t_g)$ 中，对于 $M=3$，四比特的 GHZ 态的酉算符要求 $\chi_{0,1}(t_g)=\chi_{0,2}(t_g)=\chi_{0,3}(t_g)=\chi_{1,2}(t_g)=\chi_{1,3}(t_g)=\chi_{2,3}(t_g)=\pi/4$，从而我们可以设置如下 `gatePair`形式：

In [ ]:
# Define the gate pair of general Molmer-Sorensen gate
gatePair = ([0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3])

通过调用 `pulseGeneralMS()` 函数，我们可以生成四比特 GHZ 态所需的脉冲以及对应的酉矩阵以及相应门的失真度：

In [ ]:
res, ureal = pulseGeneralMS(gatePair, args1=args1, args2=args2)
print("The generated unitary gate is:\n", np.round(ureal, 2))
print(f"The parallel Mølmer-Sørensen gate infidelity:\n {res['infidelity']}")

为了直观的显示广义 MS 门的效果，用户可以计算广义 MS 门作用后计算基矢上的末态的分布。我们首先需要从量脉中导入相关的生成基矢以及画图的包：

In [ ]:
# Import the function for qubit basis
from Quanlse.Utils.Functions import basis
from Quanlse.Utils.Functions import computationalBasisList
# Import the function for plot bar figures
from Quanlse.Utils.Plot import plotBarGraph

然后通过广义 MS 门生成的酉矩阵，我们可以计算运行完后的末态，并计算末态的布局数：

In [ ]:
finalState = (ureal @ np.array(basis(16, 0))).T[0] # Get final state
population = [abs(state ** 2) for state in finalState] # calulate the states distribution
basis = computationalBasisList(4, 2) # define the qubit basis list

通过量脉内置的画图包，用户可以直观的显示 GHZ 态的布局数：

In [ ]:
plotBarGraph(basis, population, "Population of a 4-Qubits GHZ state generated by General Mølmer-Sørensen gate",
             "Computational Basis", "Population")

可以看到，生成的末态中，态主要分布在 $|0000\rangle$ 和 $|1111\rangle$ 两个多比特计算基矢上，符合 GHZ 态的分布规律。

## 总结

通过量脉离子阱，用户可以方便地生成并行的 MS 所需的激光脉冲，更进一步，得益于离子阱全连通的特性，用户可以借助量脉支持的广义 MS 门脉冲生成功能，方便地实现多粒子的 GHZ 态，并通过量脉可视化地显示离子阱广义 MS 门生成的 GHZ 态布局数。

## 参考文献

\[1\] [Figgatt, Caroline, et al. "Parallel entangling operations on a universal ion-trap quantum computer." Nature 572.7769 (2019): 368-372.](https://www.nature.com/articles/s41586-019-1427-5)

\[2\] [Lu, Yao, et al. "Global entangling gates on arbitrary ion qubits." Nature 572.7769 (2019): 363-367.](https://www.nature.com/articles/s41586-019-1428-4)